In [1]:
import json
import re

VCDB_FILE_JSON = "data/joined/vcdb_1-of-1.json"
VCDB_EXCEL_OUTPUT = "vcdb_regression.xlsx"

NAIC_CODES = {
    "11": "Agriculture, Forestry, Fishing and Hunting",
    "21": "Mining",
    "22": "Utilities",
    "23": "Construction",
    "3": "Manufacturing",
    "42": "Wholesale Trade",
    "44": "Retail Trade",
    "45": "Retail Trade",
    "48": "Transportation and Warehousing",
    "49": "Transportation and Warehousing",
    "51": "Information",
    "52": "Finance and Insurance",
    "53": "Real Estate Rental and Leasing",
    "54": "Professional, Scientific, and Technical Services",
    "55": "Management of Companies and Enterprises",
    "56": "Administrative and Support and Waste... Services",
    "61": "Educational Services",
    "62": "Health Care and Social Assistance",
    "71": "Arts, Entertainment, and Recreation",
    "72": "Accommodation and Food Services",
    "81": "Other Services (except Public Administration)",
    "92": "Public Administration",
}


def get_industry(d):
    industry_code = d["victim"]["industry"]
    if industry_code[0:2] in NAIC_CODES:
        return NAIC_CODES[industry_code[0:2]]
    if industry_code[0] in NAIC_CODES:
        return NAIC_CODES[industry_code[0]]
    return "Unknown"

# filter for identified victim
def filter_for_victim_id(data):
    filtered = []
    for idx, d in enumerate(data):
        if "victim_id" in d["victim"] and d["victim"]["victim_id"] != "Unknown":
            filtered.append(d)
    return filtered

# filter for recorded loss in dollars
def filter_for_recorded_loss(data):
    filtered = []
    for idx, d in enumerate(data):
        if "impact" in d and ("overall_amount" in d["impact"] or "overall_min_amount" in d["impact"] or "overall_max_amount" in d["impact"]):
            if "iso_currency_code" in d["impact"] and d["impact"]["iso_currency_code"] != "USD":
                  continue
            filtered.append(d)
    return filtered

def average_from_range(text):
    """
    Checks if the string contains a pattern of the form "[number] to [number]",
    and returns the average of those two numbers.
    
    Args:
    text (str): The input string.
    
    Returns:
    float: The average of the two numbers, or None if no match is found.
    """
    # Define the regular expression pattern to match "[number] to [number]"
    pattern = r"(\d+)\s*to\s*(\d+)"
    
    # Search for the pattern in the input text
    match = re.search(pattern, text)
    
    if match:
        # Extract the numbers from the match
        num1 = int(match.group(1))
        num2 = int(match.group(2))
        
        # Calculate the average of the two numbers
        average = (num1 + num2) / 2
        
        return int(average)
    else:
        return None  # Return None if no match is found


def filter_for_victim_employee_count(data):
    filtered = []
    for idx, d in enumerate(data):
        if "victim" in d and "employee_count" in d["victim"]:
            employee_count = d["victim"]["employee_count"]
            if employee_count == "Unknown":
                continue
            # average_from_range(d["victim"]["employee_count"])
            count = average_from_range(employee_count)
            if  count == None:
                if employee_count == "Over 100000":
                    count = 100000
                elif employee_count == "Large":
                    count = 5000
                elif employee_count == "Small":
                    count = 500
                else:
                    print(d["victim"]["employee_count"])
            d["victim"]["employee_count"] = count
            filtered.append(d)
    return filtered
    
    

def add_assets(d, entry):
    varieties = [v["variety"] for v in d["asset"]["assets"]]
    types = ["Media", "Network", "Person", "Server", "Term", "User Dev", "Embedded", "Unknown", "Other"]
    for t in types:
        entry["asset.{}".format(t)] = False

    for val in varieties:
        if val in ["Unknown", "Other"]:
            entry["asset.{}".format(val)] = True
            continue
        for t in types:
            if val.startswith(t[0]):
                entry["asset.{}".format(t)] = True
                break

def add_discovery_method(d, entry):
    methods = ["internal", "external", "partner", "other", "unknown"]
    for method in methods:
        val = method in d["discovery_method"]
        entry["discover_method.{}".format(method)] = val

def add_actor(d, entry):
    actors = ["internal", "external", "partner", "unknown"]
    for actor in actors:
        val = actor in d["actor"]
        entry["actor.{}".format(actor)] = val

def add_action(d, entry):
    actions = ["hacking", "malware", "social", "error", "misuse", "physical", "environmental", "unknown"]
    for action in actions:
        val = action in d["action"]
        entry["action.{}".format(action)] = val

def add_industry(d, entry):
    for _, ind in NAIC_CODES.items():
        entry["industry.{}".format(ind)] = False
    entry["industry.Unknown"] = False
    ind = get_industry(d)
    entry["industry.{}".format(ind)] = True



def add_attribute(d, entry):
    fields = ["confidentiality", "integrity", "availability", "unknown", "confidentiality.data_total", "availability.duration"]
    for f in fields:
        entry["attribute.{}".format(f)] = 0
    for k, v in d["attribute"].items():
        entry["attribute.{}".format(k)] += 1
        if k == "confidentiality" and "data_total" in v:
            entry["attribute.confidentiality.data_total"] += v["data_total"]

        if k == "availability":
            if "unit" not in v or v["unit"] in ["Seconds", "Never", "Unknown", "NA"]:
                continue
            if v["unit"] == "Minutes":
                entry["attribute.availability.duration"] += (v["value"] / 60)
            if v["unit"] == "Hours":
                entry["attribute.availability.duration"] += v["value"]
            if v["unit"] == "Days":
                entry["attribute.availability.duration"] += (v["value"]*24)
            if v["unit"] == "Weeks":
                entry["attribute.availability.duration"] += (v["value"]*24*7)
            if v["unit"] == "Months":
                entry["attribute.availability.duration"] += (v["value"]*24*7*4)
            if v["unit"] == "Years":
                entry["attribute.availability.duration"] += (v["value"]*52*7*24)

def transform(d):
    entry = {
        # Victim Attributes
        "victim_id": d["victim"]["victim_id"],
        "incident_year": d["timeline"]["incident"]["year"],
        "employee_count": d["victim"]["employee_count"],

        # Y value
        "loss_amount": d["impact"]["overall_amount"],
    }
    # so far good results with action and assets only
    # Add X values
    # add_actor(d, entry)
    add_action(d, entry)
    add_assets(d, entry)
    # add_discovery_method(d, entry)
    # add_industry(d, entry)
    # add_attribute(d, entry)
    return entry

with open(VCDB_FILE_JSON, "r") as f:
    data = json.load(f)

print(f"Total number of entries: {len(data)}")

data = filter_for_recorded_loss(data)
print(f"Number of entries with recorded loss: {len(data)}") # X remain

data = filter_for_victim_employee_count(data)
print(f"Number of entries with recorded loss && employee count: {len(data)}") # X remain

data = [d for d in data if get_industry(d) == "Finance and Insurance"]
entries = []
for d in data:
    entries.append(transform(d))

import pandas as pd

entries = sorted(entries, key=lambda item: item["victim_id"])

df = pd.DataFrame(entries)

for key in entries[0].keys():
    if key != "victim_id" and key != "industry":
        df[key] = df[key].astype(int)

# to excel
# Move Y column to 2nd col
y = df["loss_amount"]
df = df.drop('loss_amount', axis='columns')
df.insert(1, 'loss_amount', y)
df.to_excel(VCDB_EXCEL_OUTPUT, sheet_name='NewSheet', index=False)

# split X and Y, drop victim id
df_x = df.drop("victim_id", axis='columns')
y = df["loss_amount"]
df_x = df_x.drop('loss_amount', axis='columns')


COLUMN_NAMES = df_x.columns.tolist()
# print(COLUMN_NAMES)
# print(df_x)



Total number of entries: 9911
Number of entries with recorded loss: 276
Number of entries with recorded loss && employee count: 208


In [16]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np
from scipy import stats

def pca(X, n_components):
    """
    Performs Principal Component Analysis (PCA) on the given data.

    Args:
        X (numpy.ndarray): The input data matrix of shape (n_samples, n_features).
        n_components (int): The number of principal components to retain.

    Returns:
        numpy.ndarray: The reduced data matrix of shape (n_samples, n_components).
    """
    
    # 1. Center the data
    X_mean = np.mean(X, axis=0)
    X_centered = X - X_mean

    # 2. Compute the covariance matrix
    covariance_matrix = np.cov(X_centered, rowvar=False)

    # 3. Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # 4. Sort the eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # 5. Select the top n_components eigenvectors
    selected_eigenvectors = sorted_eigenvectors[:, :n_components]
    selected_eigenvalues = sorted_eigenvalues[:n_components]

    # print(selected_eigenvectors)
    # 7. Calculate explained variance
    total_variance = np.sum(sorted_eigenvalues)
    explained_variance = selected_eigenvalues / total_variance

    # print(f"total: {total_variance}, captured: {explained_variance}")


    # 6. Project the data onto the new subspace
    X_reduced = np.dot(X_centered, selected_eigenvectors)

    return X_reduced, selected_eigenvectors

###  Basic regression in cleartext ###
def basic_regression(matrix, Y):
    # Extract the second column (index 1) and save it to a separate array
    # Y = matrix[:, 1]

    # Remove the second column from the original matrix
    # X = np.delete(matrix, 1, axis=1)
    X = matrix

    # Compute X^T X
    XT_X = np.dot(X.T, X)

    # Compute the inverse of X^T X
    XT_X_inv = np.linalg.inv(XT_X)

    # Compute X^T Y
    XT_Y = np.dot(X.T, Y)
    
    # print("XT_X:", XT_X)
    # print("XT_Y:", XT_X)
    # print("XT_X_inv:", XT_X_inv)

    # Compute beta = (X^T X)^-1 X^T Y
    beta = np.dot(XT_X_inv, XT_Y)

    Y_pred = np.dot(X, beta)
    ssr = 0
    for i in range(len(Y_pred)):
        ssr += (Y_pred[i] - Y[i]) ** 2
    print(ssr, len(Y_pred))
    r2 = r2_score(Y, Y_pred)


    return beta, r2

def pretty_print_model(model):
    data = {
        "Model": ["OLS"],
        "No. Observations": [model["Number of Observations"]],
        "Df Residuals": [model["Df Residuals"]],
        "Df Model": [model["Df Model"]],
    }
    print("\n===============================================================")
    df = pd.DataFrame(data)
    print(df)

    data = {
        "R-squared": model["R-Squared"],
        "Adj. R-squared": model["Adjusted R-Squared"],
        "F-statistic": model["F-statistic"],
        "Prob (F-statistic)": model["Probability (F-statistic)"],
    }
    print("===============================================================")
    df = pd.DataFrame(data)
    print(df)

    
    
    data = {
        "coef": [ format(x[0], '.4e') for x in model["Beta Coefficients"]],
        "std err": [ format(x, '.4e') for x in model["Standard Error"]],
        't': [ x[0] for x in model["t-Stat"]],
        'P>|t|': [ x[0] for x in model['P>|t|']],
    }

    df = pd.DataFrame(data)
    print("===============================================================")
    print(df)

    print("===============================================================\n")
    

PRIVATE_DATA_SRCX = df_x.to_numpy()
PRIVATE_DATA_SRCY = y
NUM_FIRMS = len(df_x)
NUM_DIM = 5

def get_private_data(firm_idx):
    return PRIVATE_DATA_SRCX[firm_idx], PRIVATE_DATA_SRCY[firm_idx]

def private_decrypt(firm_idx, cipher):
    return cipher

def encrypt(x, pubK):
    return x

def combine_decryptions(ciphers):
    """
    Agg = ciphers[0]
    for i in range(1, len(ciphers)):
        Agg += ciphers[i]
    """
    return ciphers[0]

def typical_analysis(X, Y):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    # print(f"Scaled X: {X}")
    
    df_red, eigen = pca(X, NUM_DIM)
    # print(f"Eigen vectors: {eigen}")

    Y = np.array(Y)
    Y = Y.reshape((len(X), 1))
    
    df_red = np.hstack((np.ones((len(Y), 1)), df_red))

    import statsmodels.api as sm
    model = sm.OLS(Y, df_red)
    results = model.fit()
    print(results.summary())

def decrypt(cipher, num_firms):
    partial_decrypt = []
    
    for i in range(0, num_firms):
        partial_decrypt.append(private_decrypt(i, cipher))
    
    Agg = combine_decryptions(partial_decrypt)
    return Agg

from enum import Enum

class AggComputation(Enum):
    MEAN = 1
    STD_ERR = 2
    COVARIANCE = 3

def private_compute(firm_idx, step, pk, parameters):
    x, y = get_private_data(firm_idx)
    # add constant parameter
    x = np.hstack((1, x))
    
    ### STEP 0 ###
    if step == 0:
        return encrypt(x, pk)

    ### STEP 1 ###
    if step == 1:
        means = parameters["means"]
        diff = []
        for i in range(len(x)):
            diff.append((x[i] - means[i])**2)
        return encrypt(np.array(diff), pk)

    ### STEP 2 ###
    if step == 2:
        means, sigmas = parameters["means"], parameters["sigmas"]
        z = []
        # skip first (constant) parameter
        for i in range(1, len(x)):
            if sigmas[i] == 0:
                z.append(0)
            else:
                z.append((x[i] - means[i])/ sigmas[i])
        z = np.array(z).reshape((1, len(z)))
        return encrypt(np.dot(z.T, z), pk)

    ### STEP 3 ###
    if step == 3:
        means, sigmas, eigen = parameters["means"], parameters["sigmas"], parameters["eigens"]
        z = []
        # skip first (constant) parameter
        for i in range(1, len(x)):
            if sigmas[i] == 0:
                z.append(0)
            else:
                z.append((x[i] - means[i])/ sigmas[i])
        z = np.array(z).reshape((1, len(z)))
        red_x = np.dot(z, eigen)
        red_x = np.hstack((np.array(1).reshape(1,1), red_x))
        
        return (encrypt(np.dot(red_x.T, red_x), pk), encrypt(np.dot(red_x.T, y), pk), encrypt(np.dot(y.T, y), pk))                

def OLS_regression(XT_X, XT_Y, YT_Y):
    # Compute the inverse of X^T X
    XT_X_inv = np.linalg.inv(XT_X)

    # Compute beta = (X^T X)^-1 X^T Y
    beta = np.dot(XT_X_inv, XT_Y)
    xb = np.dot(XT_X[0], beta)
    B_BT = np.dot(beta, beta.T)
    B_BT_XT_X = np.dot(B_BT, XT_X)
    sum_xb2 = np.sum(np.diagonal(B_BT_XT_X.T))
    no_obvs = int(XT_X[0][0])

    sum_sq_resid = YT_Y - (2 * np.dot(XT_Y.T, beta)) + sum_xb2
    sum_sq_resid = sum_sq_resid[0]
    df_resid = no_obvs - (NUM_DIM + 1)
    mse = resid_var = sum_sq_resid / df_resid
    df_reg = NUM_DIM 
    reg_sum_sq =  sum_xb2 - (2 * (XT_Y[0] / no_obvs) * xb) + (no_obvs * ((XT_Y[0] / no_obvs) ** 2))
    reg_sum_sq = reg_sum_sq[0]
    msr = reg_sum_sq / df_reg
    f_stat = msr / mse
    
    tss = sum_sq_resid + reg_sum_sq
    f_sig  = stats.f.sf(f_stat, df_reg, df_resid)
    r_sq = 1 - (sum_sq_resid / tss)
    
    std_err = (resid_var * np.diagonal(XT_X_inv)) ** 0.5
    t_stat = []
    p_val = []
    for i, coef in enumerate(beta):
        t_val = coef/std_err[i]
        t_stat.append(t_val)
        p_val.append(2 * stats.t.sf(abs(t_val), no_obvs - NUM_DIM- 1))

    t_stat = np.array(t_stat)
    p_val = np.array(p_val)


    adjusted_rsq = 1 - ((1 - r_sq) * (no_obvs - 1) / (no_obvs - NUM_DIM - 1))

    model = {
        "Beta Coefficients" : beta,
        "Standard Error": std_err.T,
        "t-Stat": t_stat,
        "P>|t|": p_val,
        "Df Residuals": df_resid,
        "Df Model": df_reg,
        "Number of Observations": no_obvs,
        "R-Squared": r_sq,
        "Adjusted R-Squared": adjusted_rsq,
        "F-statistic": f_stat,
        "Sum of Squared Residuals": sum_sq_resid, # good val
        "Probability (F-statistic)": f_sig,
    }
    return model


""" Takes place after key distribution """
def mpc_analysis(pubK, num_firms):

    ### STEP 0 ###
    step = 0
    Agg_c = private_compute(0, step, pubK, None)
    for i in range(1, num_firms):
        Agg_c += private_compute(i, step, pubK, None)
    sum_X = decrypt(Agg_c, num_firms)
    num_features = len(sum_X)
    num_X = sum_X[0]
    means = []
    for i in range(num_features):
        means.append(sum_X[i]/sum_X[0])


    ### STEP 1 ###
    step = 1
    Agg_c = private_compute(0, step, pubK, {"means": means})
    for i in range(1, num_firms):
        Agg_c += private_compute(i, step, pubK, {"means": means})
    Agg = decrypt(Agg_c, num_firms)
    sigmas = []
    for i in range(num_features):
        val = (Agg[i] / num_X ) ** 0.5
        sigmas.append(val)

    ### STEP 2 ###
    step = 2
    Agg_c = private_compute(0, step, pubK, {"means": means, "sigmas": sigmas})
    for i in range(1, num_firms):
        Agg_c += np.array(private_compute(i, step, pubK, {
            "means": means, 
            "sigmas": sigmas
        }))

    Cov = decrypt(Agg_c, num_firms)
    eigen_vals, eigen_vects = np.linalg.eig(Cov)

    # Sort the eigenvalues and eigenvectors in descending order, 
    # and select the top NUM_DIM eigenvectors
    sorted_indices = np.argsort(eigen_vals)[::-1]
    sorted_eigenvalues = eigen_vals[sorted_indices]
    sorted_eigenvectors = eigen_vects[:, sorted_indices]

    selected_eigenvectors = sorted_eigenvectors[:, :NUM_DIM]
    selected_eigenvalues = sorted_eigenvalues[:NUM_DIM]

    ### STEP 3 ###
    step = 3
    Agg_covx, Agg_covxy, Agg_covy = private_compute(0, step, pubK, {
        "means": means,
        "sigmas": sigmas,
        "eigens": selected_eigenvectors
    })
    for i in range(1, num_firms):
        covx, covxy, covy = private_compute(i, step, pubK, {
            "means": means, 
            "sigmas": sigmas, 
            "eigens": selected_eigenvectors
        })
        Agg_covx += covx
        Agg_covxy += covxy
        Agg_covy += covy
    
    XT_X, XT_Y, YT_Y = ( 
        decrypt(Agg_covx, num_firms),
        decrypt(Agg_covxy, num_firms),
        decrypt(Agg_covy, num_firms)
    )
    # perform the regression
    model = OLS_regression(XT_X, XT_Y, YT_Y)

    return model, selected_eigenvectors
    

# typical_analysis(df_x, y)
# model, eigens = mpc_analysis(df_x, NUM_FIRMS)
# pretty_print_model(model)





##############################################################################
# get relative feature weights for first two components

PCAs = [[], []]
for i in range(2):
    vector = np.abs(eigens[:, i])
    total_val = np.sum(vector)
    for j in range(len(COLUMN_NAMES)):
        val = (vector[j] / total_val)
        PCAs[i].append(val)

relative_weights = {
    "feature name": COLUMN_NAMES,
    "Relative weight - PC 1": PCAs[0],
    "Relative weight - PC 2": PCAs[1],
}

sorted_indices = np.argsort(PCAs[0])[::-1]
pca1 = {
    "feature name": [COLUMN_NAMES[i] for i in sorted_indices],
    "Relative weight - PC 1": [PCAs[0][i] for i in sorted_indices],
}

sorted_indices = np.argsort(PCAs[1])[::-1]
pca2 = {
    "feature name": [COLUMN_NAMES[i] for i in sorted_indices],
    "Relative weight - PC 2": [PCAs[1][i] for i in sorted_indices],
}


rel = pd.DataFrame(relative_weights)
print(rel)
print()
rel = pd.DataFrame(pca1)
print(rel)
print()
rel = pd.DataFrame(pca2)
print(rel)
print()


# get relative feature weights for first two components
beta = model["Beta Coefficients"]
contributions = [0] * len(COLUMN_NAMES)
for i in range(2):
    vector = eigens[:, i]
    # total_val = np.sum(vector)
    for j in range(len(COLUMN_NAMES)):
        val = vector[j] * beta[i+1]
        contributions[j] += val[0]

sorted_indices = np.argsort(contributions)[::-1]
feat_coef = {
    "feature name": [COLUMN_NAMES[i] for i in sorted_indices],
    "coef": [format(contributions[i], '.4e') for i in sorted_indices]
}
rel = pd.DataFrame(feat_coef)
print(rel)
print()

################################################################################






        



            feature name  Relative weight - PC 1  Relative weight - PC 2
0          incident_year                0.102681                0.067092
1         employee_count                0.004173                0.063176
2         action.hacking                0.074171                0.012674
3         action.malware                0.043338                0.104992
4          action.social                0.073752                0.174587
5           action.error                0.038368                0.063199
6          action.misuse                0.061336                0.071062
7        action.physical                0.171989                0.054300
8   action.environmental                0.000000                0.000000
9         action.unknown                0.000000                0.000000
10           asset.Media                0.006575                0.024519
11         asset.Network                0.000000                0.000000
12          asset.Person                0.073752   